In [2]:
import os
import sys

In [3]:
IN_COLAB='google.colab' in sys.modules

In [28]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR/artigo/unicamp_IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    TREC_EVAL="/content/drive/MyDrive/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval"
else:
    WORKING_FOLDER="/work/ia368v_dd/unicamp_IR/"
    TREC_EVAL="/work/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval"

In [5]:
import pandas as pd
import numpy as np

In [6]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [7]:
ALL_EVALUATIONS="tests/evaluation_20_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20240615.tsv"

BM25_RUN="tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt"
BM25_E5_RUN="tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt"
BM25_MT5_RUN="tests/200-queries-passagens-by-BM25-mT5_v4_with_query_id_run.txt"
SPLADE_RUN="tests/splade_pt_result_with_clueweb_id_run.txt"
COLBERTX_RUN="tests/colbertx_all_znormalized_validation_scores_000_199_run.txt"

E5_RUN="tests/e5_docs_000_999_50_sampled_queries_run.txt"
E5_BASE_RUN="tests/e5_base_docs_000_999_50_sampled_queries_run.txt"
SPLADE_MT5_RUN="tests/splade_mt5_docs_000_099_50_sampled_queries_run.txt"

BM25_1M_RUN="tests/bm25_1M_docs_000_999_sampled_50_queries_run.txt"
BM25_1M_MASSIVEWEB_RUN="tests/bm25_massiveweb_1M_docs_000_999_sampled_50_queries_run.txt"

COLBERTX_1M_MASSIVEWEB_RUN="tests/colbertx_massiveweb_1M_docs_000_999_sampled_50_queries_run.txt"

E5_COLBERTX_RRF_RUN="tests/colbertx_e5_RRF_000_999_sampled_50_queries_run.txt"
E5_SPLADE_RRF_RUN="tests/splade_e5_fixed_RRF_000_999_sampled_50_queries_run.txt"

ADAV2_RUN="tests/adav2_1M_docs_000_999_sampled_50_queries_run.txt"

BM25_MT5_1M_MASSIVEWEB_RUN="tests/bm25_mt5_1M_docs_fixed_000_099_sampled_50_queries_run.txt"

COLBERTX_1M_MASSIVEWEB_RUN="tests/colbertx_massiveweb_1M_docs_000_999_sampled_50_queries_run.txt"

BM25_E5_1M_MASSIVEWEB_RUN="tests/bm25_e5_1M_docs_fixed_000_999_sampled_50_queries_run.txt"

TEXT_EMBEDDING_3_LARGE_1M_MASSIVEWEB_RUN="tests/text-embedding-3-large_1M_docs_000_999_sampled_50_queries_run.txt"

TEXT_EMBEDDING_3_SMALL_1M_MASSIVEWEB_RUN="tests/text-embedding-3-small_1M_docs_000_999_sampled_50_queries_run.txt"

In [8]:
TEXT_EMBEDDING_3_SMALL_1M_MASSIVEWEB="tests/text-embedding-3-small_1M_docs_000_999_sampled_50_queries.tsv"
TEXT_EMBEDDING_3_SMALL_HALF_1M_MASSIVEWEB="tests/text-embedding-3-small_half_normalized_1M_docs_000_999_sampled_50_queries.tsv"

In [73]:
DATASET_1M_FIELDS=['bm25_1M_massiveweb', 
                   'colbertx_1M_massiveweb', 
                   'adav2', 
                   'bm25_mT5_1M_massiveweb_fix', 
                   'bm25_e5_1M_massiveweb', 
                   'text-embbeding-3-large_1M_massiveweb',
                   'text-embbeding-3-small_1M_massiveweb',
                   'text-embbeding-3-small_half_1M_massiveweb']

In [74]:
def prepare_run_format(results_filename, add_index=True, score_field='score', invert_score=False, retriever_name="retriever"):

    results_df = pd.read_csv(results_filename, sep='\t')

    display(results_df.head())

    results_grouped_df = results_df.groupby("query_id")['passage_id'].count()

    how_many_passages_per_question = results_grouped_df.iloc[0]

    if invert_score:
        print("Inverting score values in '{}' field...".format(score_field))

        results_df['score_inv'] = 1 - results_df[score_field]
        score_field = 'score_inv'

    run_df = results_df.sort_values(["query_id", score_field], ascending=[True, False])

    run_df['Q0'] = "Q0"
    run_df['comment'] = retriever_name

    if add_index:
        run_df['index'] = np.tile(list(range(1, how_many_passages_per_question + 1)), results_grouped_df.shape[0])

    output_filename = "{}_run.txt".format(os.path.splitext(results_filename)[0])

    print("output_filename={}".format(output_filename))

    run_df[['query_id', 'Q0', "passage_id", "index", score_field, "comment"]].to_csv(output_filename, header=None, index=False, sep=" ")

    return output_filename, run_df

In [75]:
retrievers_results = []

In [76]:
retrievers_1M_results = []

### Format GPT-4 evaluations to qrels format

In [77]:
qrels_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))
qrels_1M_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_1M_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))

In [79]:
if os.path.exists(qrels_filename) == False:
    eval_df = pd.read_csv(ALL_EVALUATIONS, sep='\t')

    eval_df

    eval_df.drop_duplicates("query_id")[['query_id', 'query']]

    #### Check scores distribution on the current qrels

    np.unique(eval_df['score'].to_numpy(), return_counts=True)

    for group_name, group_df in eval_df.groupby('query_id'):
        print("\nquery_id={}".format(group_name))

        print(np.unique(group_df['score'].to_numpy(), return_counts=True))

    eval_df.groupby('query_id').count()

    eval_df.groupby('query_id').count()['query'].describe()

    #### Prepare the qrels format for trec_eval tools

    eval_df['0'] = '0'

    print("\n\n")
    print(qrels_filename)
    print(qrels_1M_filename)

    eval_df.sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_filename, header=None, index=False, sep=" ")


    #### Select 1M dataset evaluations

    eval_df[eval_df[DATASET_1M_FIELDS].sum(axis=1) > 0].sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_1M_filename, header=None, index=False, sep=" ")

else:
    print("QRELS files ({}) already created...".format(qrels_filename))


query_id=1
(array([0, 1, 2, 3]), array([22, 50, 19, 10]))

query_id=2
(array([0, 1, 2, 3]), array([120,   2,   1,   2]))

query_id=9
(array([0, 1, 2, 3]), array([74, 10, 16,  6]))

query_id=11
(array([0, 1, 2, 3]), array([59, 21,  8,  2]))

query_id=13
(array([0, 1, 2, 3]), array([45, 17, 25,  5]))

query_id=15
(array([0, 1, 2, 3]), array([40, 23,  4, 17]))

query_id=17
(array([0, 1, 2, 3]), array([37, 11,  2, 39]))

query_id=20
(array([0, 1, 2, 3]), array([18, 70, 32,  8]))

query_id=21
(array([0, 1, 2, 3]), array([56, 25, 13, 22]))

query_id=22
(array([0, 1, 2, 3]), array([62, 17,  2,  7]))

query_id=24
(array([0, 1, 2, 3]), array([69, 38, 14,  2]))

query_id=26
(array([0, 1, 2, 3]), array([14, 21, 16, 34]))

query_id=28
(array([0, 1, 2, 3]), array([64, 25, 14,  4]))

query_id=47
(array([0, 1, 2, 3]), array([61, 35, 35,  3]))

query_id=49
(array([0, 1, 2, 3]), array([33, 34, 24,  2]))

query_id=51
(array([0, 1, 2, 3]), array([43, 24, 14,  1]))

query_id=54
(array([0, 1, 2, 3]), arra

## 10M dataset

### Compute BM25 nDCG@10

In [31]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt

In [32]:
retrievers_results.append({'retriever':'BM25', 'nDCG@10': result[0].split('\t')[-1]})

In [33]:
retrievers_results

[{'retriever': 'BM25', 'nDCG@10': '0.4467'}]

### Compute BM25 + E5 nDCG@10

In [34]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_E5_RUN}

In [35]:
retrievers_results.append({'retriever':'BM25+E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 nDCG@10

In [36]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_MT5_RUN}

In [37]:
retrievers_results.append({'retriever':'BM25+mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE nDCG@10

In [38]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_RUN}

In [39]:
retrievers_results.append({'retriever':'SPLADEv2_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERTX nDCG@10

In [40]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_RUN}

In [41]:
retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 nDCG@10

In [42]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_RUN}

In [43]:
retrievers_results.append({'retriever':'E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 base nDCG@10

In [44]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_BASE_RUN}

In [45]:
retrievers_results.append({'retriever':'E5 base', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE + mT5 nDCG@10

In [46]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_MT5_RUN}

In [47]:
retrievers_results.append({'retriever':'SPLADEv2_pt-BR + mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 + ColBERT-X RRF nDCG@10

In [48]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_COLBERTX_RRF_RUN}

In [49]:
retrievers_results.append({'retriever':'E5 + ColBERT-X mMARCO_pt-BR RRF', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 + SPLADE RRF nDCG@10

In [50]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_SPLADE_RRF_RUN}

In [51]:
retrievers_results.append({'retriever':'E5 + SPLADEv2_pt-BR RRF', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 1M nDCG@10

In [52]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_1M_RUN}

retrievers_results.append({'retriever':'BM25 1M', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 1M massiveweb filtered nDCG@10

In [53]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'BM25 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERT-X 1M massiveweb filtered nDCG@10

In [54]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ADAv2 1M massiveweb filtered nDCG@10

In [55]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {ADAV2_RUN}

retrievers_results.append({'retriever':'ADA v2 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 1M massiveweb filtered nDCG@10

In [56]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_MT5_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'BM25+mT5 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + E5 1M massiveweb filtered nDCG@10

In [57]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_E5_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'BM25+E5 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute text-embedding-3-large 1M massiveweb filtered nDCG@10

In [58]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {TEXT_EMBEDDING_3_LARGE_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'text-embedding-3-large 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare text-embedding-3-small 1M massiveweb filtered run and compute nDCG@10

In [59]:
text_embedding_3_small_filename, _ = prepare_run_format(TEXT_EMBEDDING_3_SMALL_1M_MASSIVEWEB, retriever_name="text-embedding-3-small")

,query_id,query,passage_id,colbertx_id,position,score,passage
0,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-46-05303_17,7028401,1,0.676163,"http://www.klimanaturali.org/2014/03/biomas-e-fitofisionomias-brasileiros.html 8/17 f15/03/2018 BIOMAS E FITOFISIONOMIAS BRASILEIROS: CARACTERÍSTICAS E EVOLUÇÃO DA FAUNA E FLORA - Klima Naturali™ colonização no Brasil e os ciclos de desenvolvimento do país SITE  BRASIL  MUNDO  CIÊNCIAS  GERAL  levaram o homem a ocupar e destruir parte desse espaço. Fauna e Flora Rica em diversidade de espécies, a Mata Atlântica está entre Pesquisa no Site...  as cinco regiões do mundo com maior número de espécies endêmicas. Os animais podem ser considerados generalistas ou especialistas. Os generalistas apresentam hábitos alimentares variados, alta taxa de crescimento e dispersão; vivem em áreas de vegetação aberta e secundária, tolerantes e capazes de aproveitar diferentes recursos oferecidos pelo meio ambiente. Os animais especialistas são mais exigentes em relação aos habitats nos quais vivem, com dieta especí ca, uma alteração no meio ambiente exige dos animais especialistas a procura de novos habitats. Grupos da Mata Atlântica Mamíferos – A Mata Atlântica possui cerca de 250 espécies de mamíferos, das quais 55 são endêmicas. Os mamíferos são os"
1,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-01-18874_0,4875435,2,0.667840,"Conheça 11 lindos animais típicos da fauna brasileira / Incrível Conheça 11 lindos animais típicos da fauna brasileira 4--4 Compartilhe no Facebook Compartilhe no Facebook Compartilhe no Pinterest Que o Brasil é gigante, todos nós sabemos e até já demos aqui exemplos chocantes sobre isso. Mas um outro exemplo dessa grandiosidade é quando sabemos que o país abriga 17% das espécies de aves e 10% das de anfíbios e mamíferos de todo o mundo. Neste post, o Incrível.club vai te mostrar uma lista de animais típicos do país. Mico-leão dourado © Nadine Doerle / Pixabay Este macaco “ruivo” ficou famoso ao tornar-se um símbolo de espécies brasileiras em extinção. O cuidado para a preservação da espécie, que é nativa da Mata Atlântica e aparece na nota de 20 reais, começou nos anos 1970 e se intensificou duas décadas depois. Embora a situação tenha melhorado, o risco de extinção ainda existe. Tamanduá-bandeira © joelfotos / Pixabay Este gigante que ama comer formigas — pode devorar até 30 mil por dia, inclusive cupins — é também um forte: com seus 2,20 metros, consegue até brigar com uma onça pintada. E vencer. Embora ele exista originalmente nas três Américas, já foi extinto em países como Estados Unidos e em Estados brasileiros como Espírito Santo e Rio e Janeiro. Onça pintada"
2,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-63-02968_1,2990727,3,0.661703,"A classificação dos biomas segue um padrão mundial, que identifica os grandes conjuntos de ecossistemas relacionados de acordo com condições de clima e solo específicas. No Brasil, o IBGE é o órgão responsável pela classificação dos biomas, o último estudo foi realizado em 2014. Mapa e características dos Biomas Brasileiros Mapa dos Biomas Brasileiros. (Foto: Educa Mais Brasil) Biomas Brasileiros Área Aproximada (km²) Área / Total Brasil Bioma Amazônia 4.196.943 49,29% Bioma Cerrado 2.036.448 23,92% Bioma Mata Atlântica 1.110.182 13,04% Bioma Caatinga 844.453 9,92% Bioma Pampa 176.496 2,07% Bioma Pantanal 150.355 1,76% Bioma Amazônia É o maior bioma do Brasil e ocupa os estados do Acre, Amapá, Amazonas, Pará, Roraima, parte de Rondônia, Mato Grosso, Maranhão e Tocantins. Ele também se estende por nove países da América do Sul (Bolívia, Colômbia, Equador, Venezuela, Guiana, Guiana Francesa, Peru, Suriname). O Bioma Amazônia possui um flora rica com mais 30 mil espécies de plantas. A fauna corresponde a cerca de 20% de todas as espécies do planeta, são cerca de 2 milhões de animais, sendo os principais representantes o jabuti, o jacaré, a sucuri, a onça-pintada e a arara

output_filename=tests/text-embedding-3-small_1M_docs_000_999_sampled_50_queries_run.txt


In [60]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {text_embedding_3_small_filename}

retrievers_results.append({'retriever':'text-embedding-3-small 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare text-embedding-3-small_half normalized 1M massiveweb filtered run and compute nDCG@10

In [61]:
text_embedding_3_small_half_filename, _ = prepare_run_format(TEXT_EMBEDDING_3_SMALL_HALF_1M_MASSIVEWEB, retriever_name="text-embedding-3-small_half_normalized")

,query_id,query,passage_id,colbertx_id,position,score,passage
0,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-01-18874_0,4875435,1,0.691713,"Conheça 11 lindos animais típicos da fauna brasileira / Incrível Conheça 11 lindos animais típicos da fauna brasileira 4--4 Compartilhe no Facebook Compartilhe no Facebook Compartilhe no Pinterest Que o Brasil é gigante, todos nós sabemos e até já demos aqui exemplos chocantes sobre isso. Mas um outro exemplo dessa grandiosidade é quando sabemos que o país abriga 17% das espécies de aves e 10% das de anfíbios e mamíferos de todo o mundo. Neste post, o Incrível.club vai te mostrar uma lista de animais típicos do país. Mico-leão dourado © Nadine Doerle / Pixabay Este macaco “ruivo” ficou famoso ao tornar-se um símbolo de espécies brasileiras em extinção. O cuidado para a preservação da espécie, que é nativa da Mata Atlântica e aparece na nota de 20 reais, começou nos anos 1970 e se intensificou duas décadas depois. Embora a situação tenha melhorado, o risco de extinção ainda existe. Tamanduá-bandeira © joelfotos / Pixabay Este gigante que ama comer formigas — pode devorar até 30 mil por dia, inclusive cupins — é também um forte: com seus 2,20 metros, consegue até brigar com uma onça pintada. E vencer. Embora ele exista originalmente nas três Américas, já foi extinto em países como Estados Unidos e em Estados brasileiros como Espírito Santo e Rio e Janeiro. Onça pintada"
1,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-46-05303_17,7028401,2,0.683400,"http://www.klimanaturali.org/2014/03/biomas-e-fitofisionomias-brasileiros.html 8/17 f15/03/2018 BIOMAS E FITOFISIONOMIAS BRASILEIROS: CARACTERÍSTICAS E EVOLUÇÃO DA FAUNA E FLORA - Klima Naturali™ colonização no Brasil e os ciclos de desenvolvimento do país SITE  BRASIL  MUNDO  CIÊNCIAS  GERAL  levaram o homem a ocupar e destruir parte desse espaço. Fauna e Flora Rica em diversidade de espécies, a Mata Atlântica está entre Pesquisa no Site...  as cinco regiões do mundo com maior número de espécies endêmicas. Os animais podem ser considerados generalistas ou especialistas. Os generalistas apresentam hábitos alimentares variados, alta taxa de crescimento e dispersão; vivem em áreas de vegetação aberta e secundária, tolerantes e capazes de aproveitar diferentes recursos oferecidos pelo meio ambiente. Os animais especialistas são mais exigentes em relação aos habitats nos quais vivem, com dieta especí ca, uma alteração no meio ambiente exige dos animais especialistas a procura de novos habitats. Grupos da Mata Atlântica Mamíferos – A Mata Atlântica possui cerca de 250 espécies de mamíferos, das quais 55 são endêmicas. Os mamíferos são os"
2,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-63-02968_1,2990727,3,0.671061,"A classificação dos biomas segue um padrão mundial, que identifica os grandes conjuntos de ecossistemas relacionados de acordo com condições de clima e solo específicas. No Brasil, o IBGE é o órgão responsável pela classificação dos biomas, o último estudo foi realizado em 2014. Mapa e características dos Biomas Brasileiros Mapa dos Biomas Brasileiros. (Foto: Educa Mais Brasil) Biomas Brasileiros Área Aproximada (km²) Área / Total Brasil Bioma Amazônia 4.196.943 49,29% Bioma Cerrado 2.036.448 23,92% Bioma Mata Atlântica 1.110.182 13,04% Bioma Caatinga 844.453 9,92% Bioma Pampa 176.496 2,07% Bioma Pantanal 150.355 1,76% Bioma Amazônia É o maior bioma do Brasil e ocupa os estados do Acre, Amapá, Amazonas, Pará, Roraima, parte de Rondônia, Mato Grosso, Maranhão e Tocantins. Ele também se estende por nove países da América do Sul (Bolívia, Colômbia, Equador, Venezuela, Guiana, Guiana Francesa, Peru, Suriname). O Bioma Amazônia possui um flora rica com mais 30 mil espécies de plantas. A fauna corresponde a cerca de 20% de todas as espécies do planeta, são cerca de 2 milhões de animais, sendo os principais representantes o jabuti, o jacaré, a sucuri, a onça-pintada e a arara

output_filename=tests/text-embedding-3-small_half_normalized_1M_docs_000_999_sampled_50_queries_run.txt


In [62]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {text_embedding_3_small_half_filename}

retrievers_results.append({'retriever':'text-embedding-3-small_half normalized 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [63]:
pd.DataFrame(retrievers_results).sort_values("nDCG@10", ascending=False)

,retriever,nDCG@10
2,BM25+mT5,0.7109
7,SPLADEv2_pt-BR + mT5,0.6966
8,E5 + ColBERT-X mMARCO_pt-BR RRF,0.6377
1,BM25+E5,0.6364
4,ColBERT-X_mMARCO_pt-BR,0.6279
9,E5 + SPLADEv2_pt-BR RRF,0.6272
3,SPLADEv2_pt-BR,0.5806
5,E5,0.5563
14,BM25+mT5 1M massiveweb,0.5310
16,text-embedding-3-large 1M massiveweb,0.5179


## 1M dataset

### Compute BM25 1M massiveweb filtered nDCG@10

In [80]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {BM25_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'BM25 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERT-X massiveweb filtered nDCG@10

In [81]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {COLBERTX_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ADA v2 massiveweb filtered nDCG@10

In [82]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {ADAV2_RUN}

retrievers_1M_results.append({'retriever':'ADA v2 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 1M massiveweb filtered nDCG@10

In [83]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {BM25_MT5_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'BM25+mT5 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + E5 1M massiveweb filtered nDCG@10

In [84]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {BM25_E5_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'BM25+E5 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute text-embedding-3-large 1M massiveweb filtered nDCG@10

In [85]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {TEXT_EMBEDDING_3_LARGE_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'text-embedding-3-large 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute text-embedding-3-small 1M massiveweb filtered nDCG@10

In [86]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {TEXT_EMBEDDING_3_SMALL_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'text-embedding-3-small 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute text-embedding-3-small_half 1M massiveweb filtered nDCG@10

In [87]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {text_embedding_3_small_half_filename}

retrievers_1M_results.append({'retriever':'text-embedding-3-small_half normalized 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [88]:
pd.DataFrame(retrievers_1M_results).sort_values('nDCG@10', ascending=False)

,retriever,nDCG@10
3,BM25+mT5 1M massiveweb,0.6584
5,text-embedding-3-large 1M massiveweb,0.6305
6,text-embedding-3-small 1M massiveweb,0.5676
7,text-embedding-3-small_half normalized 1M massiveweb,0.5657
2,ADA v2 1M massiveweb,0.5618
4,BM25+E5 1M massiveweb,0.5415
1,ColBERT-X_mMARCO_pt-BR 1M massiveweb,0.4918
0,BM25 1M massiveweb,0.3982
